In [1]:
# Imports
import kipoi
import os
import numpy as np
import pandas as pd

import pickle
from importlib import reload

In [9]:
!kipoi test "5UTR_Model"

INFO [kipoi.data] successfully loaded the dataloader 5UTR_Model/. from /data/ouga04b/ag_gagneur/home/karollus/5UTRModel/Collab/kipoi/5UTR_Model/dataloader.py::StrandedSequenceVariantDataloader
INFO [kipoi.model] Downloading model arguments weights from https://zenodo.org/record/3374833/files/utr_model_avgpool.h5?download=1
Using downloaded and verified file: /data/ouga04b/ag_gagneur/home/karollus/5UTRModel/Collab/kipoi/5UTR_Model/downloaded/model_files/weights/1bac703c830d6322dbace78721b1e405
Using TensorFlow backend.
2019-08-26 17:01:06.000173: I tensorflow/core/platform/cpu_feature_guard.cc:141] Your CPU supports instructions that this TensorFlow binary was not compiled to use: SSE4.1 SSE4.2 AVX AVX2 FMA
2019-08-26 17:01:06.009370: I tensorflow/core/platform/profile_utils/cpu_utils.cc:94] CPU Frequency: 2095185000 Hz
2019-08-26 17:01:06.011560: I tensorflow/compiler/xla/service/service.cc:150] XLA service 0x55f8e12ba180 executing computations on platform Host. Devices:
2019-08-26 17:

In [2]:
# Source model
model = kipoi.get_model("5UTR_Model", source="dir")

Using downloaded and verified file: /data/ouga04b/ag_gagneur/home/karollus/5UTRModel/Collab/kipoi/5UTR_Model/downloaded/model_files/weights/1bac703c830d6322dbace78721b1e405


Using TensorFlow backend.


Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.
Instructions for updating:
Use tf.cast instead.
Instructions for updating:
Deprecated in favor of operator or tf.math.divide.


In [3]:
# Path of the vcf file
vcf_path = "TestFiles/stopLoss.vcf.gz"

# Path of the fasta file
fasta_path = "/s/genomes/human/hg19/fasta/hg19.fa"
# Set true if fasta has no chr prefix, false otherwise
num_chr = False

# Path of the bed file specifying human 5utr
# If chr prefix is present, use:
# gencodev19_5utr_sorted.bed, else:
# gencodev19_5utr_sorted_noprefix.bed
bed_path = "TestFiles/gencodev19_5utr_sorted.bed"
id_column = 4

# 
#chr_order_file = "TestFiles/chrom_order_noprefix.txt"

# output file path
output_file_path = "patho.tsv"

# id map path
id_map_path = "TestFiles/hg19_idmap.tsv"

In [4]:
model.pipeline.predict_to_file(output_file_path, {"intervals_file":bed_path, 
                               "fasta_file":fasta_path,
                               "vcf_file":vcf_path,
                               "id_column":id_column,
                               "num_chr":num_chr},
                              batch_size=64)

/data/ouga04b/ag_gagneur/home/karollus/.conda/envs/karollus-env/lib/python3.6/site-packages/kipoiseq/dataloaders/sequence.py:88: FutureWarning: read_table is deprecated, use read_csv instead.
  sep='\t')
/data/ouga04b/ag_gagneur/home/karollus/.conda/envs/karollus-env/lib/python3.6/site-packages/kipoiseq/dataloaders/sequence.py:100: FutureWarning: read_table is deprecated, use read_csv instead.
  sep='\t')
100%|██████████| 1/1 [00:04<00:00,  4.98s/it]


In [5]:
# Load data as dataframe and merge with gene names
df = pd.read_csv(output_file_path, sep="\t")
df = df.rename(index=str, columns={"metadata/chr":"chr",
          "metadata/exon_positions":"exon_positions",
          "metadata/id":"transcript_id",
          "metadata/strand":"strand",
          "metadata/variants":"variants",
          "preds/mrl_fold_change":"mrl_fold_change",
          "preds/shift_1":"shift_1",
          "preds/shift_2":"shift_2"}
)

# Id map
df_map = pd.read_csv(id_map_path, sep="\t")
df = df.merge(df_map, on="transcript_id")

In [6]:
df

,chr,exon_positions,transcript_id,strand,variants,mrl_fold_change,shift_1,shift_2,gene_id,gene_name
0,chrX,68048839-68049619,ENST00000204961,+,chrX:68049525:T:['C'],-0.081899,0.106978,-0.115071,ENSG00000090776,EFNB1
1,chr4,159593490-159593608,ENST00000307738,+,chr4:159593534:A:['G'],-0.140904,-0.059372,0.193931,ENSG00000171503,ETFDH
2,chr8,21986683-21986723;21987940-21988565,ENST00000312841,-,chr8:21988118:T:['C'],0.102993,-0.070834,-0.041693,ENSG00000168453,HR
3,chr17,29421944-29422327,ENST00000356175,+,chr17:29422055:A:['C'],-0.084574,-0.048338,0.237754,ENSG00000196712,NF1
4,chr17,29421944-29422327,ENST00000358273,+,chr17:29422055:A:['C'],-0.084574,-0.048338,0.237754,ENSG00000196712,NF1
5,chr8,21986683-21986723;21987940-21989381,ENST00000381418,-,chr8:21988118:T:['C'],0.004778,-0.007438,-0.002313,ENSG00000168453,HR
6,chr17,29421994-29422327,ENST00000431387,+,chr17:29422055:A:['C'],-0.086739,-0.049234,0.256214,ENSG00000196712,NF1
7,chr4,159593276-159593608,ENST00000511912,+,chr4:159593534:A:['G'],-0.032477,-0.029689,-0.015868,ENSG00000171503,ETFDH
